In [555]:
import pandas as pd
import numpy as np
import math
import os
import re

In [505]:
N_MOVIES = 1000
N_USERS = 10000

In [506]:
def get_input_matrix():
    '''
    Get the input matrix

    Return
    ----------
    (data, W): (np.array(N_USERS, N_MOVIES), np.array(N_USERS, N_MOVIES))
        The input array with the true ratings and Nan where no ratings where given and the 
        array containing 1 where the entries are given and 0 otherwise.
    '''

    data_pd = pd.read_csv('../data/data_train.csv') 

    # get users, movies
    users, movies = [np.squeeze(arr) 
                    for arr in np.split(data_pd.Id.str.extract('r(\d+)_c(\d+)').values.astype(int) - 1, 2, axis=-1)]
    # get predictions
    predictions = data_pd.Prediction.values

    # create data matrix
    data = np.full((N_USERS, N_MOVIES), np.nan)
    W = np.full((N_USERS, N_MOVIES), 0)

    # populate data matrix
    for user, movie, pred in zip(users, movies, predictions): 
        data[user][movie] = pred
        W[user][movie] = 1 if not math.isnan(pred) else 0
    
    return (data, W)

In [507]:
X, W = get_input_matrix()

In [508]:
X.shape

(10000, 1000)

In [509]:
def compute_cosine_similarity(X, user=True):
    '''
    Compute the cosine similarity between every pair of users or items
    
    Parameters
    ----------
    X : np.array(N_USERS, N_MOVIES)
        The matrix with ratings.
        
    user : bool, default True
           a boolean which says whether we want to compute the cosine similarity between every users or 
           between every items.

    Return
    ----------
    similarity: np.array(N_USERS, N_USERS) if user = True else np.array(N_MOVIES, N_MOVIES):
                The similarity score (between -1 and 1 if X has negative values, where -1 means two vectors 
                going in the opposite direction or between 0 and 1 if we only have positive values where 0 means
                orthogonal vectors) for each user-user or item-item pair. The returned matrix is therefore 
                symmetric.
    '''
    X = np.nan_to_num(X) # Replace Nan by 0 (the dot product will hence be 0, what we want)
    
    if not user:
        X = X.T
    
    similarity = np.zeros((X.shape[0], X.shape[0]))
    for i,user in enumerate(X):
        
        similarity[i, :] = (X@user)/(np.linalg.norm(X, axis=1)*np.linalg.norm(user))
    
    return similarity

In [510]:
def compute_pearson_correlation_coefficient(X, user=True, statistic_to_use="mean"):
    '''
    Compute the pearson correlation coefficient between every pair of users or items
    
    Parameters
    ----------
    X : np.array(N_USERS, N_MOVIES)
        The matrix with ratings.
        
    user : bool, default True
           a boolean which says whether we want to compute the cosine similarity between every users or 
           between every items.
    statistic_to_use: String, either 'mean' or 'median', default 'mean'
                      The method use to center the data points.

    Return
    ----------
    similarity: np.array(N_USERS, N_USERS) if user = True else np.array(N_MOVIES, N_MOVIES):
                The similarity score (between -1 and 1) for each user-user or item-item pair.
                The returned matrix is therefore symmetric.
    '''
    if not user:
        X = X.T
    
    if statistic_to_use == "mean":
        statistic = np.nanmean(X, axis=1)
    elif statistic_to_use == "median":
        statistic = np.nanmedian(X, axis=1)
    else:
        raise ValueError(f"{statistic_to_use} is not a valid statistic! Should be 'mean' or 'median'")
    
    centered_X = X-statistic.reshape(-1,1)
    
    return compute_cosine_similarity(centered_X, user=True) # Always True since we have already taken the transpose in this method

In [511]:
def compute_SigRA(X, W, user=True):
    '''
    Compute the SiGra (https://ieeexplore.ieee.org/document/8250351) similarity between every pair of users or 
    items. Note that this method already uses weighting and hence should not be followed by the 
    similarity_weighting function.
    
    Parameters
    ----------
    X : np.array(N_USERS, N_MOVIES)
        The matrix with ratings.
    
    W : np.array(N_USERS, N_MOVIES):
        The mask containing 1 for rated items and 0 for unrated items.
        
    user : bool, default True
           a boolean which says whether we want to compute the cosine similarity between every users or 
           between every items.

    Return
    ----------
    similarity: np.array(N_USERS, N_USERS) if user = True else np.array(N_MOVIES, N_MOVIES):
                The similarity score for each user-user or item-item pair.
                The returned matrix is therefore symmetric.
    '''    
    if not user:
        X = X.T
        W = W.T
    
    similarity = np.zeros((X.shape[0], X.shape[0]))

    number_ratings = np.sum(W, axis=1)
    
    for i, (uw, ux) in enumerate(zip(W, X)):
        for j, (vw, vx) in enumerate(zip(W, X)):
            common_ratings = np.logical_and(uw, vw)
            number_common_ratings = np.sum(common_ratings)
            if number_common_ratings == 0:
                similarity[i, j] = 0
            else:
                ratios_sum = np.sum(np.minimum(ux[common_ratings], vx[common_ratings])/np.maximum(ux[common_ratings], vx[common_ratings]))
                weight = 1.0/(1+np.exp(-(number_ratings[i] + number_ratings[j])/(2*number_common_ratings))) #Why number_common_ratings in the denominator? Would make more sense to inverse numerator and denominator, but like that in the paper
                similarity[i, j] = weight*ratios_sum/number_common_ratings
    
    return similarity

In [512]:
def similarity_weighting(similarity, W, method="weighting", threshold=10):
    '''
    Weight the similarity matrix based on the number of ratings of each entry. Without weighting, users having 
    just few entries are often considered as closer, which this method tries to prevent.
    
    Parameters
    ----------
    similarity : np.array(N_USERS, N_USERS) or np.array(N_MOVIES, N_MOVIES)
                 The matrix with similarity between users or movies
        
    W : np.array(N_USERS, N_MOVIES):
        The mask containing 1 for rated items and 0 for unrated items.
        
    method: String, either 'weighting', 'significance' or 'sigmoid', default 'weighting'
                      The method use to weight the similarity.
                      Weighting weights all entries based on the number of common rated items and number of rated items.
                      Significance only reduce importance when number of common rated items is below the threshold.
                      Sigmoid reduce weight when users have only few common rated items. It keeps most of the similarity
                      measure almost untouched and hence is the softest weighting method.
    
    threshold: int, default 10
               Only used if method is 'significance'. Minimum number of common rated items needed to not have
               a decrease in importance. Needs to be adapted depending on the number of common users/items. For
               user-based similarity, should be around 7, for item-based similarity, around 70. 
               TODO: read paper to have more knowledge about what a good threshold is.

    Return
    ----------
    weighted_similarity: np.array(N_USERS, N_USERS) or np.array(N_MOVIES, N_MOVIES) depending of shape of similarity:
                The weighted similarity score (between -1 and 1) for each user-user or item-item pair.
                The returned matrix is therefore symmetric.
    '''
    assert (similarity.shape[0] == W.shape[0] or similarity.shape[0] == W.shape[1]) and similarity.shape[0] == similarity.shape[1]
    
    weighted_similarity = np.zeros_like(similarity)
    if similarity.shape[0] != W.shape[0]:
        W=W.T # We were using the items and not the users for the similarity
    
    number_ratings = np.sum(W, axis=1)
    
    for i, u in enumerate(W):
        for j, v in enumerate(W):
            number_common_ratings = np.sum(np.logical_and(u, v))
            
            if method == "weighting":
                weight = 2*number_common_ratings/(number_ratings[i] + number_ratings[j]) if (number_ratings[i] + number_ratings[j]) != 0 else 0
            elif method == "significance":
                weight = np.minimum(number_common_ratings, threshold)/threshold
            elif method == "sigmoid":
                weight = 1.0/(1+np.exp(-number_common_ratings/2))
            else:
                raise ValueError(f"{method} is not a valid method! Should be 'weighting', 'significance' or 'sigmoid'")

            weighted_similarity[i, j] = weight*similarity[i, j]
    
    return weighted_similarity

In [56]:
similarity = compute_cosine_similarity(X)

In [514]:
similarity_items = compute_cosine_similarity(X, False)

In [93]:
compute_pearson_correlation_coefficient(X, statistic_to_use="median")

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ...,  1., -1., -1.],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, -1.]])

In [98]:
similarity_items_pearson=compute_pearson_correlation_coefficient(X, False, statistic_to_use="median")

In [103]:
similarity_items_pearson

array([[ 1.        , -0.02009841,  0.04217509, ...,  0.02842744,
         0.0526932 ,  0.03357856],
       [-0.02009841,  1.        , -0.01603365, ..., -0.02378553,
         0.00502976,  0.02908714],
       [ 0.04217509, -0.01603365,  1.        , ...,  0.05456939,
         0.04203638, -0.01824104],
       ...,
       [ 0.02842744, -0.02378553,  0.05456939, ...,  1.        ,
         0.08803759,  0.01628506],
       [ 0.0526932 ,  0.00502976,  0.04203638, ...,  0.08803759,
         1.        ,  0.03081192],
       [ 0.03357856,  0.02908714, -0.01824104, ...,  0.01628506,
         0.03081192,  1.        ]])

In [105]:
similarity_items

array([[1.        , 0.09305098, 0.1306107 , ..., 0.08812148, 0.15839492,
        0.12179982],
       [0.09305098, 1.        , 0.11384168, ..., 0.08535064, 0.11111508,
        0.14269472],
       [0.1306107 , 0.11384168, 1.        , ..., 0.10391016, 0.13360448,
        0.1435644 ],
       ...,
       [0.08812148, 0.08535064, 0.10391016, ..., 1.        , 0.16871144,
        0.14879859],
       [0.15839492, 0.11111508, 0.13360448, ..., 0.16871144, 1.        ,
        0.20594874],
       [0.12179982, 0.14269472, 0.1435644 , ..., 0.14879859, 0.20594874,
        1.        ]])

In [109]:
weighted_similarity = weight_similarity(similarity_items, W)

In [152]:
significance_similarity = similarity_weighting(similarity_items, W, method="significance", threshold=7)

In [515]:
sigmoid_similarity = similarity_weighting(similarity_items, W, method="sigmoid")

In [183]:
similartiry_items_SiGra = compute_SigRA(X, W, user=False)

In [193]:
similartiry_items_SiGra

array([[0.73105858, 0.72079709, 0.73050547, ..., 0.68383528, 0.72625998,
        0.78522803],
       [0.72079709, 0.73105858, 0.68952065, ..., 0.67338861, 0.74773731,
        0.75743673],
       [0.73050547, 0.68952065, 0.73105858, ..., 0.71954167, 0.76983944,
        0.69953118],
       ...,
       [0.68383528, 0.67338861, 0.71954167, ..., 0.73105858, 0.77151787,
        0.71935246],
       [0.72625998, 0.74773731, 0.76983944, ..., 0.77151787, 0.73105858,
        0.75868866],
       [0.78522803, 0.75743673, 0.69953118, ..., 0.71935246, 0.75868866,
        0.73105858]])

In [553]:
def weighted_average_predict(X, W, similarity, k=10, with_std = False, verbose=True):
    '''
    Predict the missing values by a weighted average of the ratings of the k nearest neighbors with a weight 
    corresponding to their similarity. Take into account the mean value of the user (respectively item)
    
    
    Parameters
    ----------
    X : np.array(N_USERS, N_MOVIES)
        The matrix with ratings.
    
    W : np.array(N_USERS, N_MOVIES):
        The mask containing 1 for rated items and 0 for unrated items.
    
    similarity : np.array(N_USERS, N_USERS) or np.array(N_MOVIES, N_MOVIES)
                 The matrix with similarity between users or movies
        
    k: int, default 10
       Number of nearest neighbors
    
    with_std: bool, default False
              If set to true, take into account the std to compute a Z-score when computing the weights

    verbose: bool, default True:
             If set to True, print update messages
    
    Return
    ----------
    predictions: np.array(N_USERS, N_MOVIES)
                 The predictions for the missing values
    '''
    was_transposed=False
    printing_interval=200
    
    if similarity.shape[0] != W.shape[0]: # We were using the items and not the users for the similarity 
        was_transposed=True
        W=W.T   
        X=X.T
        printing_interval=30
    
    predictions = X.copy()

    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            if not W[i, j]:
                user_mean = np.nanmean(X[i, :])
                possible_neighbors = np.where(W[:, j])[0]
                sorted_possible_neighbors = possible_neighbors[np.flip(np.argsort(similarity[i, possible_neighbors]))]
                nearest_neighbors = sorted_possible_neighbors[:k] 
                
                if with_std:
                    number_items_rated = np.sum(W[i, :])
                    
                    user_std = np.sqrt(np.sum((X[i, W[i, :]]-user_mean)**2)/(number_items_rated-1)) if number_items_rated > 1 else 1
                    neighbors_means = np.nanmean(X[nearest_neighbors, :], axis=1)
                    
                    neighbors_number_item_rated = np.sum(W[nearest_neighbors, :], axis=1)
                    neighbors_number_item_rated[neighbors_number_item_rated<=1]=2 #To avoid division by 0 problems, should not happen frequently, set std to 1 later
                    
                    neighbors_stds = np.sqrt(np.sum((X[nearest_neighbors, :][W[nearest_neighbors, :]]-neighbors_means)**2, axis=1)/(neighbors_number_item_rated-1))
                    neighbors_stds[neighbors_number_item_rated<=1] = 1 #Set std to 1 if it was the only rating
                    
                    predictions[i, j] = user_mean + user_std * np.sum(np.multiply(similarity[i, nearest_neighbors], (X[nearest_neighbors, j]-np.nanmean(X[nearest_neighbors, :], axis=1))/neighbors_stds))/np.sum(similarity[i, nearest_neighbors])
                else:  
                    predictions[i, j] = user_mean + np.sum(np.multiply(similarity[i, nearest_neighbors], X[nearest_neighbors, j]-np.nanmean(X[nearest_neighbors, :], axis=1)))/np.sum(similarity[i, nearest_neighbors])
                
        if verbose and (i==X.shape[0]-1 or (not i%printing_interval and i!=0)):
            similarity_type = "user" if not was_transposed else "item"
            print(f"Done with {similarity_type} {i}/{X.shape[0]}")
            
    return predictions.T if was_transposed else predictions

In [554]:
pred = weighted_average_predict(X, W, sigmoid_similarity)

Done with row 30/1000
Done with row 60/1000
Done with row 90/1000
Done with row 120/1000
Done with row 150/1000
Done with row 180/1000
Done with row 210/1000
Done with row 240/1000
Done with row 270/1000
Done with row 300/1000
Done with row 330/1000
Done with row 360/1000
Done with row 390/1000
Done with row 420/1000
Done with row 450/1000
Done with row 480/1000
Done with row 510/1000
Done with row 540/1000
Done with row 570/1000
Done with row 600/1000
Done with row 630/1000
Done with row 660/1000
Done with row 690/1000
Done with row 720/1000
Done with row 750/1000
Done with row 780/1000
Done with row 810/1000
Done with row 840/1000
Done with row 870/1000
Done with row 900/1000
Done with row 930/1000
Done with row 960/1000
Done with row 990/1000
Done with row 999/1000


In [569]:
def generate_submission(pred, submission_name="submission", compression_type ="zip"):
    sample = pd.read_csv('../data/sampleSubmission.csv') # load sample submission
    sample = sample.astype({"Prediction": float}, errors='raise')
    for index, row in sample.iterrows():
        r, c = re.findall(r'r(\d+)_c(\d+)', row["Id"])[0]
        sample.at[index, "Prediction"] = pred[int(r)-1][int(c)-1]
    
    sample.to_csv(submission_name, compression=compression_type, float_format='%.3f', index = None)

In [570]:
def submit_on_kaggle(name="submission.zip", message=None):
    '''
    Submit a solution on kaggle.

    Parameters
    ----------
    name: str (optional)
        name of the file to submit
    message: str (optional)
        Message to use with the submission. Makes easier to 
        understand what each submission is about
    '''
    command = f"kaggle competitions submit -c cil-collaborative-filtering-2022 -f {name}"

    if not message is None:
        command = command + f" -m {message}"

    os.system(command)

In [568]:
generate_submission(pred, submission_name="sigmoid_cosine_similarity_items_10nn")

In [571]:
submit_on_kaggle(name="sigmoid_cosine_similarity_items_10nn", message="Similarity based using items, cosine similarity with sigmoid weighting and 10 nearest neighbors")

In [559]:
pred

array([[3.48457718, 3.46575115, 3.29454848, ..., 2.79584825, 2.99129092,
        3.33164412],
       [3.34920169, 2.47014907, 2.84239086, ..., 5.        , 3.        ,
        3.        ],
       [2.27405046, 3.02060159, 2.87631228, ..., 2.10074353, 2.37203414,
        2.56367779],
       ...,
       [2.91337448, 3.36292369, 3.19491513, ..., 2.76052163, 2.91109604,
        3.4841525 ],
       [2.80468284, 3.44195667, 3.54328259, ..., 2.66757643, 2.83965279,
        3.04405327],
       [2.87667227, 3.44069647, 3.30077702, ..., 2.98914642, 3.08887743,
        3.        ]])